> `Matthieu ALCARO, Emilien LAMBERT, Antonio VALLERA, Leo MARCEL, Fares QEDIRA`
---

# **ZOIDBERG 2.0**
Dans ce cahier, nous allons utiliser 3 jeux de données d'images de radiographies de poumons pour aider les médecins à détecter si le patient a une pneumonie.

Pour ce faire, nous allons d'abord classifier nos jeux de données car il s'agit de différents types d'images, certaines sont en R.G.B (3D) et d'autres en N&B (2D).
Ensuite, nous les transformerons pour déterminer la maladie pulmonaire.

 En ce qui concerne notre travail précédent sur MNIST, qui consistait à reconnaître différents chiffres manuscrits, nous avons intégré notre logique MNIST dans ce projet 'zoidberg 2.0'.


## 1. **Installation de paquets externes**

Certains paquets et modules externes sont nécessaires pour faire fonctionner l'ensemble du module, comme :

>- `matplotlib` *(Creates visualizations in Python)*
    - `matimage` *(image loading, rescaling and display operations)*
    - `pylot` *(functions to manipulate elements of a figure)*
- `numpy` *(extended mathematical functions)*
- `graphviz` *(create graph objects)*
- `sklearn` *(provides learning algorithms)*
    - `metrics`
    - `decomposition`
    - `kernel_approximation`
    - `neural_network`
    - `neighbors`
    - `naive_bayes`
    - `tree`
    - `ensemble`
    - `model_selection`
- `math` *(basic math functions)*
- `mnist` *(database of handwritten digits)*



In [ ]:
#installation of dependencies
!pip install sklearn matplotlib numpy graphviz python-mnist python-math image

import os
import re
import time

#external packages
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import graphviz

from sklearn import svm, metrics, tree
from sklearn.decomposition import PCA
from sklearn.kernel_approximation import Nystroem
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from math import sqrt

from mnist import MNIST

from datetime import datetime, timedelta

     |████████████████████████████████| 7.9MB 9.6MB/s 
  Created wheel for image: filename=image-1.5.33-py2.py3-none-any.whl size=19482 sha256=ef9069aa01ca0455c49e80092b4cdb600e19d04271aa0c8caa06dbb923ec808a
  Stored in directory: /root/.cache/pip/wheels/87/4c/60/d5904e69c837fcdea7e03ffa0c657f35ced7e398c6f3ca17cc
Successfully built image



## *2*. **Fonction pour resize / formatter les images**

#### Les images ne font pas toutes la même taille, de plus certaines sont en noir et blanc, d'autres en 3 dimensions (RGB). Nous devont donc les traiter afin de les utiliser avec différents modèles de ScikitLearn.


In [1]:
def crop(img, size):
    middleH = img.shape[0] / 2
    middleW = img.shape[1] / 2

    lowH = middleH - (size / 2)
    maxH = middleH + (size / 2)

    lowW = middleW - (size / 2)
    maxW = middleW + (size / 2)

    cropped = img[int(lowH):int(maxH), int(lowW):int(maxW)]
    return cropped

def data_formatting(path):
    # List all name of images file in a table
    images_name = [f for f in os.listdir(path) if re.match(r'.*\.jpeg', f)]

    # Create table of label and image path
    labels = []
    images_path = []
    counter = 0

    # Create a table of reformatted images
    images = []
    good_images = []
    bad_images = []

    for image_name in images_name:
        img = mpimg.imread(path + image_name)
        if img.ndim == 2:
            cropped = crop(img, 200)
            cropped = cropped / 255
            cropped = np.reshape(cropped, 40000)
            good_images.append(cropped)
            if "virus" in image_name:
                labels.append("virus")
            elif "bacteria" in image_name:
                labels.append("bacteria")
            else:
                labels.append("normal")

    return good_images, labels


## *3*. **Fonction pour charger les images (Mnist ou pneumonie selon le paramètre)**

Data Formating permet de trier les data, et de renvoyer ainsi les images avec le bon label. On charge ensuite les images avec load_dataset


In [ ]:
def data_formatting(path):
    # List all name of images file in a table
    images_name = [f for f in os.listdir(path) if re.match(r'.*\.jpeg', f)]

    # Create table of label and image path
    labels = []
    images_path = []
    counter = 0

    # Create a table of reformatted images
    images = []
    good_images = []
    bad_images = []

    for image_name in images_name:
        img = mpimg.imread(path + image_name)
        if img.ndim == 2:
            cropped = crop(img, 200)
            cropped = cropped / 255
            cropped = np.reshape(cropped, 40000)
            good_images.append(cropped)
            if "virus" in image_name:
                labels.append("virus")
            elif "bacteria" in image_name:
                labels.append("bacteria")
            else:
                labels.append("normal")
        elif img.ndim == 3:
            bad_images.append(img)

    return good_images, labels

In [ ]:
def load_dataset(type_data):
    if type_data == 'mnist':
        mn_data = MNIST('./mnist/')
        images_training, labels_training = mn_data.load_training()
        images_testing, labels_testing = mn_data.load_testing()
        data = {
            'np_images_training': np.array(images_training),
            'np_labels_training': np.array(labels_training),
            'np_images_testing': np.array(images_testing),
            'np_labels_testing': np.array(labels_testing)
        }
        return data

    elif type_data == 'pneumonia':

        train_images, train_labels = data_formatting("data ia sorted/all/train/")
        test_images, test_labels = data_formatting("data ia sorted/all/test/")
        validation_images, validation_labels = data_formatting("data ia sorted/all/validation/")
        
        data = {
            'np_images_training': np.array(train_images),
            'np_labels_training': np.array(train_labels),
            'np_images_testing': np.array(test_images),
            'np_labels_testing': np.array(test_labels),
            "np_images_validation": np.array(validation_images),
            "np_labels_validation": np.array(validation_labels),
        }
        return data


# 2. **Importation de jeux de données**

Cette partie du code sert uniquement à importer le jeu de données dans notre code, jeu de données situé dans le dossier ./mnist et "data ia sorted".

In [ ]:
    print("--------------- START LOAD_DATASET ---------------")
    start_time = time.time()
    data = load_dataset('pneumonia')
    print("--------------- FINISH : %s SECONDS ---------------" % (time.time() - start_time))
    print("--------------- DATASET HAS BEEN IMPORTED ---------------")


--------------- START LOAD_DATASET ---------------


FileNotFoundError: ignored

# 3. **Transform**

Certain algo nécessite / et ou sont plus performant avec une action de type transform sur les images.
La fonction Transform() permet de redimensionner les images afin qu'à la fin, elles est toutes les mêmes dimensions.

In [ ]:
    # Les transform
    print("--------------- START TRANSFORM ---------------")
    # start_time = time.time()
    # transform = Transformation()
    # train_data_transform = transform.nystroem(models.np_images_training)
    # test_data_transform = transform.nystroem(models.np_images_testing)
    # models.np_images_training = train_data_transform
    # models.np_images_testing = test_data_transform
    print("--------------- FINISH : %s SECONDS ---------------" % (time.time() - start_time))

# 4. **Algorithmes**

Scikit-learn est une bibliothèque libre Python destinée à l'apprentissage automatique.

**On commence d'abord par initaliser les datasets**

In [ ]:
def __init__(self, data):
        self.np_images_training = data["np_images_training"]
        self.np_labels_training = data["np_labels_training"]
        self.np_images_testing = data["np_images_testing"]
        self.np_labels_testing = data["np_labels_testing"]

**Méthode pour obtenir le score**

In [ ]:
def get_scores(self, model):
        train_score = model.score(self.np_images_training, self.np_labels_training) * 100
        test_score = model.score(self.np_images_testing, self.np_labels_testing) * 100
        print('\n\n--- Training score : %.3f' % (train_score))
        print('\n--- Testing score : %.3f \n\n' % (test_score))

Une prédiction de classe est la suivante : étant donné le modèle finalisé et une ou plusieurs instances de données, prédire la classe pour les instances de données.
Nous ne connaissons pas les classes de résultats pour les nouvelles données. C'est pourquoi nous avons besoin du modèle en premier lieu.
Nous pouvons prédire la classe des nouvelles instances de données à l'aide de notre modèle de classification finalisé dans scikit-learn en utilisant la fonction predict().
Par exemple, nos instances de données peut être transmis à la fonction predict() de notre modèle afin de prédire les valeurs de classe pour chaque instance du tableau.

In [ ]:
def get_predictions(self, model):
        return model.predict(self.np_images_testing)

Cette fonction va faire un tracé d'une matrice de confusion sklearn cm en utilisant une visualisation de carte thermique Seaborn. à montrer dans chaque carré. count : Si True, montre le nombre brut dans la matrice de confusion. La valeur par défaut est True.

In [ ]:
def get_matrix(self, model, predicted):
        print(f"Classification report for classifier {model}:\n"
              f"{metrics.classification_report(self.np_labels_testing, predicted)}\n")

        disp = metrics.plot_confusion_matrix(model, self.np_images_testing, self.np_labels_testing)
        disp.figure_.suptitle("Confusion Matrix")
        print(f"Confusion matrix:\n{disp.confusion_matrix}")
        plt.show()

## SVC

**Le SVC (Support Vector Classification) est un algorithme appartenant à l’ensemble des SVM (Support Vector Machines).** 
Le SVC prend plusieurs hyperparamètres, notamment le gamma.
Plus le gamma est grand, plus l’algorithme essaie de s’adapter aux données de façon précise. On peut d’ailleurs constater qu’augmenter le gamma de façon trop conséquente conduit à un “overfitting” et donc un modèle moins précis.

In [ ]:
def svc(self):
        clf = svm.SVC(verbose=True)
        clf.fit(self.np_images_training, self.np_labels_training)
        predicted = clf.predict(self.np_images_testing)
        print("Accuracy:", metrics.accuracy_score(self.np_labels_testing, predicted))

Le SVC Linear est un SVC avec un paramètre kernel ‘Linear’

In [ ]:
def svc_linear(self):
        clf = svm.LinearSVC(verbose=True)
        clf.fit(self.np_images_training, self.np_labels_training)
        predicted = clf.predict(self.np_images_testing)
        print("Accuracy:", metrics.accuracy_score(self.np_labels_testing, predicted))

## MLP Classifier

**Le MLP Classifier met en œuvre un algorithme de perceptron multicouche (MLP) qui s'entraîne par rétro propagation.**
MLP s'entraîne sur deux tableaux : le tableau X de taille (n_samples, n_features), qui contient les échantillons d'entraînement représentés sous forme de vecteurs de caractéristiques en virgule flottante ; et le tableau y de taille (n_samples,) qui contient les valeurs cibles (étiquettes de classe) pour les échantillons d'entraînement.


In [ ]:
def mlp_classifier(self):
        clf = MLPClassifier(verbose=True, solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(784, 3), random_state=1)
        clf.fit(self.np_images_training, self.np_labels_training)
        predicted = clf.predict(self.np_images_testing)
        print("Accuracy:", metrics.accuracy_score(self.np_labels_testing, predicted))

## Prédiction KNN 

**Nearest Neighbors met en œuvre l'apprentissage non supervisé des plus proches voisins.** Le choix de l'algorithme de recherche des voisins est contrôlé par le mot clé 'algorithm', qui doit être l'un de ['auto', 'ball_tree', 'kd_tree', 'brute']. Lorsque la valeur par défaut "auto" est utilisée, l'algorithme tente de déterminer la meilleure approche à partir des données d'apprentissage.

Pour calculer la valeur la plus adaptée du k (le nombre de voisins utilisé par l’algorithme), il faut prendre la racine carré du nombre total d’image d’entrainement (train). Dans notre cas, on laisse le choix de l’algorithme a la valeur par défaut pour utiliser le plus adapté. Cela permet d'entraîner au mieux possible l’algorithme et d’avoir une bonne précision lors de la phase de test.

In [ ]:
    def prediction_knn(self):
        # Create KNN Classifier
        k = round(sqrt(self.np_labels_training.size))  # k does be square root of the training set records
        knn = KNeighborsClassifier(n_neighbors=k)  # call the k nearest neighbors

        # Train the model using the training sets
        knn.fit(self.np_images_training, self.np_labels_training)

        # Predict the response for test dataset
        predicted = knn.predict(self.np_images_testing)

        print("Accuracy:", metrics.accuracy_score(self.np_labels_testing, predicted))

## Naive Bayes

**Les méthodes Naive Bayes sont un ensemble d'algorithmes d'apprentissage supervisé basés sur l'application du théorème de Bayes.** Le théorème de Bayes établit une relation prenant en compte la variable de classe et un vecteur de caractéristiques. 
Les classificateurs Naive Bayes ont très bien fonctionné dans de nombreuses situations du monde réel, notamment dans la classification des documents et le filtrage. Ils nécessitent une petite quantité de données d'entraînement pour estimer les paramètres nécessaires.

In [ ]:
def naive_bayes(self):
        model = GaussianNB()
        # fit the model with the training data
        model.fit(self.np_images_training, self.np_labels_training)
        
        predicted = model.predict(self.np_images_testing)
        print("Accuracy:", metrics.accuracy_score(self.np_labels_testing, predicted))

## Decision Tree Classifier

**DecisionTreeClassifier est une classe capable d'effectuer une classification multi-classes sur un ensemble de données.**
Comme les autres classificateurs, Decision Tree Classifier prend en entrée deux tableaux : un tableau d’entrainement, clairsemé ou dense, de forme (images_training, labels_training) contenant les échantillons d'apprentissage, et un tableau de test de valeurs entières, de forme (images_tests, labels_tests), contenant les étiquettes de classe pour les échantillons d'apprentissage.

In [ ]:
def decision_tree_classifier(self, max_depth):
        # max_depth = nombre de niveau dans l'arbre (+ grand = + precis (jusqu'a un certain point), - graph lisible)
        # entropy = par rapport au gain
        clf = DecisionTreeClassifier(criterion="entropy", max_depth=max_depth, min_samples_split=2, random_state=0)
        clf = clf.fit(self.np_images_training, self.np_labels_training)

        # scores = cross_val_score(clf, self.np_images_training, self.np_labels_training, cv=5)
        # print(scores.mean())

        # export to pdf the training tree classification schema
        dot_data = export_graphviz(clf, out_file=None, filled=True, rounded=True, special_characters=True)
        graph = graphviz.Source(dot_data)
        graph.render("Tree_Graph")

        predicted = clf.predict(self.np_images_testing)
        print("Accuracy:", metrics.accuracy_score(self.np_labels_testing, predicted))

## Random Tree forest

**Random Forest est un modèle d'ensemble composé de nombreux arbres de décision,
des sous-ensembles aléatoires de caractéristiques et le vote moyen pour faire des prédictions.**

Le fonctionnement de l’algorithme est donc similaire à celui du Decision Tree Classifier avec en paramètre supplémentaire (n_estimators) qui correspond au nombre d'arbres qu’il va exécuter. Plus ce paramètre sera grand plus les performances seront élevées mais le code sera ralenti.

Pour récupérer un graphique en résultat, on récupère un graphique aléatoire parmi les différents Decision Tree Classifier effectuer par le Random Tree Forest.

In [ ]:
def random_tree_forest(self, estimators, max_depth):
        # n_estimators = + le nombre est grand, + les performances seront bonnes mais le code sera ralenti
        clf = RandomForestClassifier(n_estimators=estimators, max_depth=max_depth, min_samples_split=2, random_state=0)
        clf = clf.fit(self.np_images_training, self.np_labels_training)

        # scores = cross_val_score(clf, self.np_images_training, self.np_labels_training, cv=5)
        # print(scores.mean())

        # create graph in a pdf
        # take a random tree in the forest and display it !!!
        estimator = clf.estimators_[randrange(estimators)]
        dot_data = export_graphviz(estimator, out_file=None, filled=True, rounded=True, special_characters=True)
        graph = graphviz.Source(dot_data)
        graph.render("Tree_Forest_Graph")

        # Predict the response for test dataset
        predicted = clf.predict(self.np_images_testing)
        print("Accuracy:", metrics.accuracy_score(self.np_labels_testing, predicted))

## Extremely Randomized Trees

**Le classificateur ExtraTrees (Extremely Randomized Trees) fonctionne à l'identique par rapport au Random Tree Forest.** Extra Trees teste des divisions aléatoires sur une fraction des caractéristiques (contrairement à Random Forest, qui teste toutes les divisions possibles sur une fraction des caractéristiques).

In [ ]:
def extremely_randomized_trees(self, estimators, max_depth):
        # ExtraTrees classifier always tests random splits over fraction of features
        # (in contrast to RandomForest, which tests all possible splits over fraction of features)

        clf = ExtraTreesClassifier(n_estimators=estimators, max_depth=max_depth, min_samples_split=2, random_state=0)
        clf = clf.fit(self.np_images_training, self.np_labels_training)

        # Predict the response for test dataset
        predicted = clf.predict(self.np_images_testing)
        print("Accuracy:", metrics.accuracy_score(self.np_labels_testing, predicted))

# 6. **Exécution**

Dans le main on va pouvoir selectionner le type d'algo que l'on souhaite exécuter

In [ ]:
def main():
    print("--------------- START LOAD_DATASET ---------------")
    start_time = time.time()
    data = load_dataset('pneumonia')
    print("--------------- FINISH : %s SECONDS ---------------" % (time.time() - start_time))

    models = Sklearn(data)

    # visualize = Statistics()
    # visualize.pca_3d(models.np_images_training, models.np_labels_training)

    # Les transform
    print("--------------- START TRANSFORM ---------------")
    # start_time = time.time()
    # transform = Transformation()
    # train_data_transform = transform.nystroem(models.np_images_training)
    # test_data_transform = transform.nystroem(models.np_images_testing)
    # models.np_images_training = train_data_transform
    # models.np_images_testing = test_data_transform
    print("--------------- FINISH : %s SECONDS ---------------" % (time.time() - start_time))

    # Les algo
    print("--------------- START TRAINING ---------------")
    start_time = time.time()
    # models.svc()
    # print(models.svc_linear())
    # my_model = models.mlp_classifier()

    # models.prediction_knn()
    # models.naive_bayes()
    # models.decision_tree_classifier(5)
    models.random_tree_forest(100, 5)
    # models.extremely_randomized_trees(100, 10)

    print("--------------- FINISH : %s SECONDS ---------------" % (time.time() - start_time))

    # Les stats
    # models.get_scores(my_model)
    # predictions = models.get_predictions(my_model)
    # models.get_matrix(my_model, predictions)


if __name__ == '__main__':
    main()